# Grid Search for K-Means in Initial and Latent Space

In [ ]:
import os

import threading
from threading import Lock

import numpy as np

from tensorflow.keras.models import load_model

from multiprocessing.pool import ThreadPool

from autoencoder import Autoencoder
from helper_funcs import *

import pandas
pandas.set_option('display.max_rows', None)

from params import kmeans_test

# Create files for later

In [ ]:
models = os.listdir('./models/')

dataset = b'MNIST/input.dat'
query   = b'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_dataset.dat'
    normalized_query   = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_query.dat'
    encoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_dataset.dat'
    encoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query]})

n = 60000

In [ ]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]

    model = b'models/' + model.encode()

    # load model
    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

# Grid Search for K-Means in Initial Space

In [ ]:
normalized_dataset = b'MNIST/model_conv_12_normalized_dataset.dat'

In [ ]:
best_params_lsh = [5, 6, 7500, 2222]          # L, k, limit_queries, window
best_params_hypercube = [2153, 13, 950, 2222] # M, k, probes, window

rows = []

In [ ]:
def print_results(clustering_time, stotal_latent, silhouette, obj_func):
    print("Clustering time       :", clustering_time)
    print("Total silhouette      :", stotal_latent)
    print("Silhouette per cluster:", silhouette)
    print("Objective function    :", obj_func)
    print("-------------------------")

def run_kmeans_classic_initial():
    config = {
        'model': b'CLASSIC',
        'vals': [],
        'dataset': normalized_dataset,
    }

    clustering_time, stotal_latent, silhouette, obj = kmeans_test(conf=config, int_data=0)

    print_results(clustering_time.value, stotal_latent.value, silhouette.val, obj.value)

    sil_val = silhouette.val

    rows.append(['CLASSIC', clustering_time.value, stotal_latent.value, sil_val, obj.value])

    del silhouette

def run_kmeans_lsh_initial():
    config = {
        'model': b'LSH',
        'vals': best_params_lsh[:-1],
        'window': best_params_lsh[-1],
        'dataset': normalized_dataset,
    }

    clustering_time, stotal_latent, silhouette, obj = kmeans_test(conf=config, int_data=0)

    print_results(clustering_time.value, stotal_latent.value, silhouette.val, obj.value)

    sil_val = silhouette.val

    rows.append(['LSH', clustering_time.value, stotal_latent.value, sil_val, obj.value])

    del silhouette

def run_kmeans_cube_initial():
    config = {
        'model': b'CUBE',
        'vals': best_params_hypercube[:-1],
        'window': best_params_hypercube[-1],
        'dataset': normalized_dataset,
    }

    clustering_time, stotal_latent, silhouette, obj = kmeans_test(conf=config, int_data=0)

    print_results(clustering_time.value, stotal_latent.value, silhouette.val, obj.value)

    sil_val = silhouette.val

    rows.append(['CUBE', clustering_time.value, stotal_latent.value, sil_val, obj.value])

    del silhouette

In [ ]:
pool = ThreadPool(processes=3)

pool.apply_async(run_kmeans_classic_initial)
pool.apply_async(run_kmeans_lsh_initial)
pool.apply_async(run_kmeans_cube_initial)

pool.close()
pool.join()

In [ ]:
col_methods, col_clustering_time, col_stotal_latent, obj_func = [], [], [], []

for row in rows:
    method, clustering_time, stotal_latent, silhouette, obj = row

    col_methods.append(method)
    col_clustering_time.append(clustering_time)
    col_stotal_latent.append(stotal_latent)
    obj_func.append(obj)

col_dict = {'method': col_methods, 'clustering time': col_clustering_time,
            'silhouette total': col_stotal_latent, 'objective function': obj_func}

df = pandas.DataFrame(data=col_dict)
df

# Grid Search for K-Means in Latent Space

In [ ]:
best_params_lsh = [4, 7, 1, 0.6]            # L, k, limit_queries, window
best_params_hypercube = [67, 3, 1000, 0.42] # M, k, probes, window

rows = []

In [ ]:
def print_results(clustering_time, stotal_latent, silhouette, obj_func):
    print("Clustering time       :", clustering_time)
    print("Total silhouette      :", stotal_latent)
    print("Silhouette per cluster:", silhouette)
    print("Objective function    :", obj_func)
    print("-------------------------")

def run_kmeans_classic_latent(model):
    encoded_dataset = model_to_files[model][2]

    config = {
        'model': b'CLASSIC',
        'vals': [],
        'dataset': encoded_dataset,
    }

    clustering_time, stotal_latent, silhouette, obj = kmeans_test(conf=config, int_data=0)

    print_results(clustering_time.value, stotal_latent.value, silhouette.val, obj.value)

    sil_val = silhouette.val

    rows.append([model, 'CLASSIC', clustering_time.value, stotal_latent.value, sil_val, obj.value])

    del silhouette

def run_kmeans_lsh_latent(model):
    encoded_dataset = model_to_files[model][2]

    config = {
        'model': b'LSH',
        'vals': best_params_lsh[:-1],
        'window': best_params_lsh[-1],
        'dataset': encoded_dataset,
    }

    clustering_time, stotal_latent, silhouette, obj = kmeans_test(conf=config, int_data=0)

    print_results(clustering_time.value, stotal_latent.value, silhouette.val, obj.value)

    sil_val = silhouette.val

    rows.append([model, 'LSH', clustering_time.value, stotal_latent.value, sil_val, obj.value])

    del silhouette

def run_kmeans_cube_latent(model):
    encoded_dataset = model_to_files[model][2]

    config = {
        'model': b'CUBE',
        'vals': best_params_hypercube[:-1],
        'window': best_params_hypercube[-1],
        'dataset': encoded_dataset,
    }

    clustering_time, stotal_latent, silhouette, obj = kmeans_test(conf=config, int_data=0)

    print_results(clustering_time.value, stotal_latent.value, silhouette.val, obj.value)

    sil_val = silhouette.val

    rows.append([model, 'CUBE', clustering_time.value, stotal_latent.value, sil_val, obj.value])

    del silhouette

In [ ]:
pool = ThreadPool(processes=4)

for model in models:
    pool.apply_async(run_kmeans_classic_latent, (model,))
    pool.apply_async(run_kmeans_lsh_latent, (model,))
    pool.apply_async(run_kmeans_cube_latent, (model,))

pool.close()
pool.join()

In [ ]:
col_models, col_methods, col_clustering_time, col_stotal_latent, obj_func = [], [], [], [], []

for row in rows:
    model, method, clustering_time, stotal_latent, silhouette, obj = row

    col_models.append(model)
    col_methods.append(method)
    col_clustering_time.append(clustering_time)
    col_stotal_latent.append(stotal_latent)
    obj_func.append(obj)

col_dict = {'model': col_models, 'method': col_methods,
            'clustering time': col_clustering_time, 'silhouette total': col_stotal_latent, 'objective function': obj_func}

df = pandas.DataFrame(data=col_dict)
df